# Convolutional Neural Network(CNN)

### 라이브러리 import

In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

### tensorflow 버전 확인

In [2]:
tf.__version__

'2.11.1'

## 파트 1 - 데이터 전처리

### Training set 전처리

In [3]:
# 이미지 데이터를 학습하기 전에 데이터 증강을 수행하는 코드
train_datagen = ImageDataGenerator(rescale = 1./255,    # 이미지의 픽셀값을 0과 1 사이의 값으로 조정
                                   shear_range = 0.2,    # 이미지를 변형시키는 시어링(Shearing) 강도를 지정(회전)
                                   zoom_range = 0.2,     # 이미지를 확대 또는 축소하는 줌(Zoom) 범위를 지정
                                   horizontal_flip = True)   # 이미지를 좌우로 뒤집는 수평방향 뒤집기를 수행

# flow_from_directory() 함수를 사용하여 디렉토리에서 이미지 데이터를 읽어와 배치(batch) 단위로 학습할 수 있도록 설정
training_set = train_datagen.flow_from_directory('dataset/training_set',   # 학습 데이터셋 디렉토리 경로를 지정
                                                 target_size = (64, 64),   # 이미지를 지정된 크기(64x64)로 리사이즈
                                                 batch_size = 32,          # 한 번에 학습할 배치(batch) 크기를 지정(이미지의 장수 32는 기본값)
                                                 class_mode = 'binary')     # 이진 분류(Binary classification) 문제로 설정


Found 8000 images belonging to 2 classes.


### Test set 전처리

데이터 증강을 수행하지 않는 이유는 모델의 일반화 성능을 평가하기 위해서이다.

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [5]:
# 각 레이어가 정확히 하나의 입력 텐서와 하나의 출력 텐서를 갖는 단일 입력 및 출력으로 간단한 모델을 만들 때 유용

cnn = tf.keras.models.Sequential()

2023-04-12 10:42:44.114229: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Step 1 - Convolution

In [6]:
# Sequential 모델에 컨볼루션 레이어를 추가 계층에는 커널 크기가 3x3인 32개의 필터가 있으며 사용된 활성화 함수는 ReLU
# 레이어의 입력 모양은 [64, 64, 3]이며 크기가 64x64이고 3개의 색상 채널(RGB)이 있는 입력 이미지

cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [7]:
# cnn에 최대 풀링 계층을 추가
# MaxPool2D 계층은 pool_size 및 strides 매개변수로 정의된 크기 창 내에서 최대값을 취하여 입력의 공간적 차원을 줄인다.
# 이 경우 pool_size=2는 풀링 작업에 2x2 윈도우가 사용됨을 의미하고 strides=2는 윈도우가 한 번에 2 단위 이동됨을 의미
# 이 계층의 출력은 입력의 다운샘플링된 버전이며 공간 차원은 2배로 줄었다.
# padding의 기본값은 valid이고 2칸씩 띄었을 때 없는 숫자 칸은 무시 할 수 있고
# padding의 same은 없는 숫자 칸은 0인 가짜 픽셀을 추가 한다.

cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [9]:
# cnn에 평탄화 레이어를 추가
# Flatten 레이어는 이전 레이어의 출력을 가져와 완전 연결 레이어로 전달할 수 있는 1차원 배열로 변환
# 이는 일반적으로 이전 레이어에서 추출한 기능 간의 관계를 학습하고 모델링할 수 있는 네트워크에 조밀한 레이어를 추가하기 전에 수행

cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [10]:
# 완전히 연결된 계층을 신경망에 추가

cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [11]:
# 완전히 연결된 계층을 신경망에 추가
# 이진 분류 문제이므로 마지막 계층의 뉴런 수는 1로 설정

cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [12]:
#  'adam' 옵티마이저는 손실 함수의 기울기에 따라 네트워크 가중치를 업데이트하는 데 사용
#  'binary_crossentropy' 손실 함수는 이진 분류 문제에 사용되며 잘못된 예측에 대해 모델에 페널티를 줌
#  학습 및 테스트 중에 모델의 성능을 평가하는 데 사용되는 메트릭은 모델에서 수행한 올바른 예측의 백분율인 정확도
#  compile() 함수는 모델을 컴파일하고 학습을 위해 준비하는 데 사용

cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [13]:
# training_set에 지정된 훈련 데이터에서 컴파일된 CNN 모델 cnn을 훈련하고 총 25 에포크 동안 test_set에 지정된 테스트 데이터에서 검증
# 교육 과정에서 모델은 Adam 옵티마이저를 사용하여 이진 교차 엔트로피 손실을 최소화하려고 시도하고 '정확도' 메트릭을 사용하여 모델의 정확도를 모니터링한다.

# 'fit()' 메서드는 모델을 교육 데|이터에 맞추고 지정된 에포크 수 동안 테스트 데이터에서 유효성을 검사하는 데 사용
# 각 에포크 동안 모델은 지정된 배치 크기(이 경우 32개)의 배치에 대해 학습되며 유효성 검사 정확도 및 손실은 각 에포크 후에 계산
# 'fit()' 메서드의 출력은 각 시대의 모델 손실 및 정확도와 같은 교육 프로세스에 대한 정보를 포함하는 'History' 개체

cnn.fit(x = training_set, validation_data = test_set, epochs = 25)

Epoch 1/25
250/250 [==============================] - 25s 95ms/step - loss: 0.6780 - accuracy: 0.5740 - val_loss: 0.6103 - val_accuracy: 0.6810
Epoch 2/25
250/250 [==============================] - 21s 83ms/step - loss: 0.6095 - accuracy: 0.6646 - val_loss: 0.5600 - val_accuracy: 0.7235
Epoch 3/25
250/250 [==============================] - 20s 81ms/step - loss: 0.5631 - accuracy: 0.7097 - val_loss: 0.5850 - val_accuracy: 0.6950
Epoch 4/25
250/250 [==============================] - 21s 84ms/step - loss: 0.5361 - accuracy: 0.7314 - val_loss: 0.5140 - val_accuracy: 0.7550
Epoch 5/25
250/250 [==============================] - 21s 84ms/step - loss: 0.5103 - accuracy: 0.7509 - val_loss: 0.5257 - val_accuracy: 0.7430
Epoch 6/25
250/250 [==============================] - 21s 85ms/step - loss: 0.4868 - accuracy: 0.7686 - val_loss: 0.5013 - val_accuracy: 0.7515
Epoch 7/25
250/250 [==============================] - 21s 84ms/step - loss: 0.4823 - accuracy: 0.7695 - val_loss: 0.4888 - val_accuracy:

## Part 4 - Making a single prediction

In [20]:
#  훈련된 CNN 모델을 사용하여 단일 이미지에 대한 예측을 하기 위한 것
import numpy as np
from keras.utils import load_img, img_to_array

# cnn모델에 입력할 크기로 조절 하면서 이미지를 불러온다.
test_image = load_img('dataset/prediction/dog.jpg', target_size = (64, 64))
# 이미지를 numpy배열로 변환
test_image = img_to_array(test_image)
# 배열의 차원을 cnn 모델의 입력 차원에 맞추기 위해 확장
test_image = np.expand_dims(test_image, axis = 0)
# cnn 모델을 사용하여 이미지에 대한 예측 수행
result = cnn.predict(test_image)
# 에측 결과를 레이블로 매핑하기 위해 훈련 데이터의 클래스 인덱스 를 가져온다.
training_set.class_indices
# 예측 결과가 1 이상인 경우 'dog' 그렇지 않은 경우 'cat'
if result[0][0] == 1:
  prediction = 'dog'
else:
  prediction = 'cat'

print(prediction)

1/1 [==============================] - 0s 24ms/step
dog
